# Training a model on CarbonTracker's carbon flux
This notebook outlines the entire workflow to load and preprocess the following data sets, to be able to train a ML model:

- CarbonTracker
- ERA5 (monthly)
- ERA5-land (monthly)
- SPEI (monthly)
- MODIS (monthly)
- Biomass (yearly)
- Copernicus Landcover (yearly)

First follow the data downloading and config setup instuctions.

If you run this notebook on Surf Research Cloud, you shouldn't need to do this anymore.

In [1]:
import datetime
from pathlib import Path

import xarray as xr
from dask.distributed import Client

import excited_workflow
from excited_workflow import carbon_tracker
from excited_workflow.source_datasets import datasets


client = Client()

Define the paths for the carbon tracker dataset, the regions dataset and create output directory, as well as define the datasets that need to be included, the input variables (x_keys) for the model and the target variable (y_key).

In [ ]:
cb_file= Path("/data/volume_2/EXCITED_prepped_data/CT2022.flux1x1-monthly.nc")
regions_file = Path("/data/volume_2/EXCITED_prepped_data/regions.nc")
output_path = Path.home()

time = datetime.datetime.now().strftime("%Y-%m-%d_%H_%M")
output_dir = output_path / f"carbon_tracker-{time}"
output_dir.mkdir(parents=True, exist_ok=True)

desired_data = [
    "biomass",
    "spei",
    "modis",
    "era5_monthly",
    "era5_land_monthly",
    "copernicus_landcover"
]

x_keys = ["d2m", "mslhf", "msshf", "ssr", "str", "t2m", "spei", "NIRv", "skt",
            "stl1", "swvl1", "lccs_class"]
y_key = "bio_flux_opt"

Merge the desired datasets into a single xr.Dataset with the same dimensions as the carbon tracker dataset. 

In [ ]:
ds_cb = xr.open_dataset(cb_file)
ds_cb = excited_workflow.utils.convert_timestamps(ds_cb)
ds_input = xr.merge(
    [
        datasets[name].load(freq="monthly", target_grid=ds_cb)
        for name in desired_data
    ]
)

To limit the analyis to Transcom region 2 (North America) we require the `regions.nc` file:

In [ ]:
df_na = carbon_tracker.mask_region(regions_file, cb_file, ds_input, 2)
df_na


bio_flux_opt  transcom_regions    biomass  \
time       latitude longitude                                              
2000-08-01 16.5     -97.5     -6.010017e-07               2.0  45.289826   
                    -96.5     -5.498427e-07               2.0  42.524410   
                    -95.5     -3.203208e-07               2.0  40.658878   
           17.5     -100.5    -7.650994e-07               2.0  46.187639   
                    -99.5     -2.389529e-06               2.0  41.231996   
...                                     ...               ...        ...   
2019-12-01 55.5     -114.5     5.078427e-07               2.0  23.803023   
                    -113.5     7.799189e-07               2.0  29.709211   
                    -112.5     8.044822e-07               2.0  15.008749   
           56.5     -131.5     5.528356e-07               2.0  64.364447   
           57.5     -132.5     1.015704e-07               2.0  13.424713   

                                   spei      NDVI      NIRv         d2m  \
time       latitude longitude                                             
2000-08-01 16.5     -97.5      0.402637  0.756299  0.264352  291.160736   
                    -96.5      0.859432  0.635423  0.186314  287.451385   
                    -95.5     -0.002091  0.714530  0.245570  292.602661   
           17.5     -100.5     0.364373  0.768887  0.270905  290.568329   
                    -99.5      0.007593  0.748444  0.249498  290.078156   
...                                 ...       ...       ...         ...   
2019-12-01 55.5     -114.5     0.650443  0.168989  0.014420  259.794739   
                    -113.5     0.582704  0.181974  0.015521  259.467133   
                    -112.5     0.328202  0.176154  0.017731  258.919037   
           56.5     -131.5    -0.181532 -0.066072 -0.038799  269.274261   
           57.5     -132.5     0.123489 -0.117813 -0.061499  266.016449   

                                    mslhf      msshf            sp  ...  \
time       latitude longitude                                       ...   
2000-08-01 16.5     -97.5     -123.552444 -29.256828  89704.265625  ...   
                    -96.5      -98.636139 -44.350342  83853.054688  ...   
                    -95.5      -88.933502 -76.770813  94853.679688  ...   
           17.5     -100.5    -118.364532 -51.205177  85797.492188  ...   
                    -99.5     -119.800644 -42.374344  85094.835938  ...   
...                                   ...        ...           ...  ...   
2019-12-01 55.5     -114.5      -2.107529  16.971649  92106.984375  ...   
                    -113.5      -1.547745  15.787460  92649.859375  ...   
                    -112.5      -0.648941  16.144135  93281.093750  ...   
           56.5     -131.5      -3.085175   0.307220  91528.421875  ...   
           57.5     -132.5      -1.342758  -2.474945  84002.898438  ...   

                                      skt        stl1        stl2        stl3  \
time       latitude longitude                                                   
2000-08-01 16.5     -97.5      293.800140  294.488708  294.490234  294.558624   
                    -96.5      291.346344  292.329987  292.415466  292.695587   
                    -95.5      298.976379  299.503906  299.532959  299.621643   
           17.5     -100.5     293.096253  293.790924  293.769348  293.723419   
                    -99.5      291.973236  292.737579  292.763458  292.868774   
...                                   ...         ...         ...         ...   
2019-12-01 55.5     -114.5     261.254547  272.136719  272.757507  274.221741   
                    -113.5     261.202423  268.530670  269.509583  273.085327   
                    -112.5     260.399170  268.575684  270.025513  273.835937   
           56.5     -131.5     265.012054  270.157807  270.336700  270.928772   
           57.5     -132.5     263.732574  267.186951  267.285309  267.666473   

                                    

Validate the model by splitting the dataset into 5 groups. Train the model over 4 groups and predict over the remaining group iteratively. Output rmse netcdfs and scatterplots are stored in the output directory.

In [ ]:
carbon_tracker.validate_model(df_na, 5, x_keys, y_key, output_dir)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.00000,0.00000,0.00000,0.77380,0.00000,4.97460,1.29000


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.00000,0.00000,0.00000,0.76720,0.00000,4.68210,1.23000


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.00000,0.00000,0.00000,0.76630,0.00000,4.55350,1.24000


2023-12-12 15:28:46,348 - distributed.utils_perf - WARNING - full garbage collections took 19% CPU time recently (threshold: 10%)
2023-12-12 15:28:47,313 - distributed.utils_perf - WARNING - full garbage collections took 22% CPU time recently (threshold: 10%)


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.00000,0.00000,0.00000,0.76210,0.00000,3.94170,1.15000


2023-12-12 15:28:52,361 - distributed.utils_perf - WARNING - full garbage collections took 20% CPU time recently (threshold: 10%)
2023-12-12 15:28:52,736 - distributed.utils_perf - WARNING - full garbage collections took 23% CPU time recently (threshold: 10%)
2023-12-12 15:28:53,320 - distributed.utils_perf - WARNING - full garbage collections took 23% CPU time recently (threshold: 10%)
2023-12-12 15:28:54,113 - distributed.utils_perf - WARNING - full garbage collections took 26% CPU time recently (threshold: 10%)
2023-12-12 15:28:54,394 - distributed.utils_perf - WARNING - full garbage collections took 26% CPU time recently (threshold: 10%)
2023-12-12 15:28:56,090 - distributed.utils_perf - WARNING - full garbage collections took 25% CPU time recently (threshold: 10%)


2023-12-12 15:28:56,572 - distributed.utils_perf - WARNING - full garbage collections took 25% CPU time recently (threshold: 10%)
2023-12-12 15:28:57,135 - distributed.utils_perf - WARNING - full garbage collections took 25% CPU time recently (threshold: 10%)
2023-12-12 15:28:58,132 - distributed.utils_perf - WARNING - full garbage collections took 25% CPU time recently (threshold: 10%)


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.00000,0.00000,0.00000,0.75800,0.00000,8.37230,1.18000


2023-12-12 15:29:03,119 - distributed.utils_perf - WARNING - full garbage collections took 23% CPU time recently (threshold: 10%)
2023-12-12 15:29:03,542 - distributed.utils_perf - WARNING - full garbage collections took 23% CPU time recently (threshold: 10%)
2023-12-12 15:29:04,091 - distributed.utils_perf - WARNING - full garbage collections took 23% CPU time recently (threshold: 10%)
2023-12-12 15:29:04,852 - distributed.utils_perf - WARNING - full garbage collections took 26% CPU time recently (threshold: 10%)
2023-12-12 15:29:05,162 - distributed.utils_perf - WARNING - full garbage collections took 26% CPU time recently (threshold: 10%)
2023-12-12 15:29:06,751 - distributed.utils_perf - WARNING - full garbage collections took 25% CPU time recently (threshold: 10%)


2023-12-12 15:29:07,223 - distributed.utils_perf - WARNING - full garbage collections took 25% CPU time recently (threshold: 10%)
2023-12-12 15:29:07,812 - distributed.utils_perf - WARNING - full garbage collections took 25% CPU time recently (threshold: 10%)


Train the model over the entire dataset.

In [ ]:
pycs, model = carbon_tracker.train_model(df_na, x_keys, y_key)

Save the model to ONNX in the output directory. 

In [ ]:

carbon_tracker.save_model(pycs, model, output_dir)

2023-12-12 15:29:08,971 - distributed.utils_perf - WARNING - full garbage collections took 26% CPU time recently (threshold: 10%)
2023-12-12 15:29:12,288 - distributed.utils_perf - WARNING - full garbage collections took 24% CPU time recently (threshold: 10%)


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.00000,0.00000,0.00000,0.76910,0.00000,3.96930,1.57000


2023-12-12 15:29:15,464 - distributed.utils_perf - WARNING - full garbage collections took 22% CPU time recently (threshold: 10%)
2023-12-12 15:29:15,987 - distributed.utils_perf - WARNING - full garbage collections took 23% CPU time recently (threshold: 10%)
2023-12-12 15:29:16,707 - distributed.utils_perf - WARNING - full garbage collections took 26% CPU time recently (threshold: 10%)
2023-12-12 15:29:17,696 - distributed.utils_perf - WARNING - full garbage collections took 26% CPU time recently (threshold: 10%)
2023-12-12 15:29:18,022 - distributed.utils_perf - WARNING - full garbage collections took 26% CPU time recently (threshold: 10%)
2023-12-12 15:29:20,130 - distributed.utils_perf - WARNING - full garbage collections took 25% CPU time recently (threshold: 10%)
The maximum opset needed by this model is only 8.


Create dataframe to run the model with. 

In [ ]:
df = df_na[x_keys]
df.head()

d2m       mslhf      msshf         ssr  \
time       latitude longitude                                                  
2000-08-01 16.5     -97.5      291.160736 -123.552444 -29.256828  17632392.0   
                    -96.5      287.451385  -98.636139 -44.350342  17875046.0   
                    -95.5      292.602661  -88.933502 -76.770813  18379498.0   
           17.5     -100.5     290.568329 -118.364532 -51.205177  17707054.0   
                    -99.5      290.078156 -119.800644 -42.374344  17862124.0   

                                     str         t2m      spei      NIRv  \
time       latitude longitude                                              
2000-08-01 16.5     -97.5     -4539167.0  294.455078  0.402637  0.264352   
                    -96.5     -5688396.0  291.743347  0.859432  0.186314   
                    -95.5     -4277693.5  298.020874 -0.002091  0.245570   
           17.5     -100.5    -3109625.0  293.402252  0.364373  0.270905   
                    -99.5     -3914028.0  293.219940  0.007593  0.249498   

                                      skt        stl1     swvl1  lccs_class  
time       latitude longitude                                                
2000-08-01 16.5     -97.5      293.800140  294.488708  0.405656        90.0  
                    -96.5      291.346344  292.329987  0.441099       120.0  
                    -95.5      298.976379  299.503906  0.340303        60.0  
           17.5     -100.5     293.096253  293.790924  0.412808        90.0  
                    -99.5      291.973236  292.737579  0.406332       120.0

Open model and run it over the dataframe to check it was saved correctly. 

In [ ]:
import datetime

from onnxruntime import InferenceSession


with open(output_dir / "lightgbm.onnx", "rb") as f:
    model = f.read()

sess = InferenceSession(model)
predictions_onnx = sess.run(None, {'X': df.to_numpy()})[0]

In [ ]:
predictions_onnx

array([[-1.1604789e-06],
       [-5.3657362e-07],
       [-4.3432399e-07],
       ...,
       [ 4.4027504e-07],
       [ 4.7860465e-07],
       [ 1.3122254e-07]], dtype=float32)